In [1]:
import logging

import nshutils as nu
import rich

logging.basicConfig(level=logging.DEBUG)

nu.pretty()

DEBUG:matplotlib:matplotlib data path: /opt/conda/envs/jmp-peft/lib/python3.11/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/root/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CONFIGDIR=/root/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/root/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /root/.cache/matplotlib/fontlist-v390.json


In [2]:
from __future__ import annotations

import os
from pathlib import Path

import mattertune.configs as MC
from mattertune import MatterTuner

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def hparams():
    hparams = MC.MatterTunerConfig.draft()

    # Model hparams
    hparams.model = MC.JMPBackboneConfig.draft()
    hparams.model.graph_computer = MC.JMPGraphComputerConfig.draft()
    hparams.model.graph_computer.pbc = True

    hparams.model.ckpt_path = Path("/mnt/shared/checkpoints/jmp-l.pt")
    hparams.model.ignore_gpu_batch_transform_error = True
    hparams.model.optimizer = MC.AdamWConfig(lr=1.0e-4)

    hparams.model.properties = []
    energy = MC.EnergyPropertyConfig(loss=MC.MAELossConfig())
    hparams.model.properties.append(energy)
    forces = MC.ForcesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(forces)
    stresses = MC.StressesPropertyConfig(loss=MC.MAELossConfig(), conservative=False)
    hparams.model.properties.append(stresses)

    # Data hparams
    hparams.data = MC.ManualSplitDataModuleConfig.draft()
    hparams.data.train = MC.OMAT24DatasetConfig.draft()
    hparams.data.train.src = Path("/mnt/datasets/salex/val/")
    hparams.data.batch_size = 1

    # Trainer hparams
    hparams.lightning_trainer_kwargs = {"fast_dev_run": True}

    rich.print(hparams)
    hparams = hparams.finalize()
    rich.print(hparams)
    return hparams


hp = hparams()
model = MatterTuner(hp).tune()
model

INFO:nshconfig._src.registry:Registered <class 'mattertune.backbones.jmp.model.JMPBackboneConfig'> with tag 'jmp'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.backbones.m3gnet.model.M3GNetBackboneConfig'> with tag 'm3gnet'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.data.omat24.OMAT24DatasetConfig'> with tag 'omat24'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.data.xyz.XYZDatasetConfig'> with tag 'xyz'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.backbones.eqV2.model.EqV2BackboneConfig'> with tag 'eqV2'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.backbones.orb.model.ORBBackboneConfig'> with tag 'orb'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.data.db.DBDatasetConfig'> with tag 'db'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.data.matbench.MatbenchDatasetConfig'> with tag 'matbench'.
INFO:nshconfig._src.registry:Registered <class 'mattertune.data.mp.MPDatasetConfig'> with tag '

MatterTunerConfig(
    lightning_trainer_kwargs={'fast_dev_run': True},
    model=JMPBackboneConfig(
        lr_scheduler=None,
        ignore_gpu_batch_transform_error=True,
        name='jmp',
        graph_computer=JMPGraphComputerConfig(
            cutoffs=CutoffsConfig(main=12.0, aeaint=12.0, qint=12.0, aint=12.0),
            max_neighbors=MaxNeighborsConfig(main=30, aeaint=20, qint=8, aint=1000),
            per_graph_radius_graph=False,
            pbc=True
        ),
        ckpt_path=Path('/mnt/shared/checkpoints/jmp-l.pt'),
        optimizer=AdamWConfig(
            name='AdamW',
            lr=0.0001,
            eps=1e-08,
            betas=(0.9, 0.999),
            weight_decay=0.01,
            amsgrad=False
        ),
        properties=[
            EnergyPropertyConfig(
                name='energy',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='energy'
            ),
            ForcesPropertyConfig(
                name='forces',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='forces',
                conservative=False
            ),
            StressesPropertyConfig(
                name='stresses',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='stresses',
                conservative=False
            )
        ]
    ),
    data=ManualSplitDataModuleConfig(
        num_workers='auto',
        pin_memory=True,
        validation=None,
        train=OMAT24DatasetConfig(type='omat24', src=Path('/mnt/datasets/salex/val')),
        batch_size=1
    )
)

MatterTunerConfig(
    data=ManualSplitDataModuleConfig(
        batch_size=1,
        num_workers='auto',
        pin_memory=True,
        train=OMAT24DatasetConfig(type='omat24', src=Path('/mnt/datasets/salex/val')),
        validation=None
    ),
    model=JMPBackboneConfig(
        properties=[
            EnergyPropertyConfig(
                name='energy',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='energy'
            ),
            ForcesPropertyConfig(
                name='forces',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='forces',
                conservative=False
            ),
            StressesPropertyConfig(
                name='stresses',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='stresses',
                conservative=False
            )
        ],
        optimizer=AdamWConfig(
            name='AdamW',
            lr=0.0001,
            eps=1e-08,
            betas=(0.9, 0.999),
            weight_decay=0.01,
            amsgrad=False
        ),
        lr_scheduler=None,
        ignore_gpu_batch_transform_error=True,
        name='jmp',
        ckpt_path=Path('/mnt/shared/checkpoints/jmp-l.pt'),
        graph_computer=JMPGraphComputerConfig(
            pbc=True,
            cutoffs=CutoffsConfig(main=12.0, aeaint=12.0, qint=12.0, aint=12.0),
            max_neighbors=MaxNeighborsConfig(main=30, aeaint=20, qint=8, aint=1000),
            per_graph_radius_graph=False
        )
    ),
    lightning_trainer_kwargs={'fast_dev_run': True}
)

CRITICAL:nshutils.typecheck:Type checking the following modules: ('jmppeft',)
INFO:jmp.models.gemnet.layers.radial_basis_dynamic_cutoff:[RadialBasis] Using absolute cutoff of 12.0 Angstroms.
INFO:jmp.models.gemnet.layers.radial_basis_dynamic_cutoff:[RadialBasis] Using absolute cutoff of 12.0 Angstroms.
INFO:jmp.models.gemnet.layers.radial_basis_dynamic_cutoff:[RadialBasis] Using absolute cutoff of 12.0 Angstroms.


Unrecognized arguments:  dict_keys(['ln', 'dropout', 'replace_scale_factors_with_ln', 'learnable_rbf', 'learnable_rbf_stds', 'unique_basis_per_layer', 'old_gaussian_implementation', 'edge_dropout'])


INFO:jmp.models.gemnet.layers.radial_basis_dynamic_cutoff:[RadialBasis] Using absolute cutoff of 12.0 Angstroms.
INFO:jmp.models.gemnet.layers.radial_basis_dynamic_cutoff:[RadialBasis] Using absolute cutoff of 12.0 Angstroms.
INFO:jmp.models.gemnet.layers.radial_basis_dynamic_cutoff:[RadialBasis] Using absolute cutoff of 12.0 Angstroms.
INFO:jmp.models.gemnet.layers.radial_basis_dynamic_cutoff:[RadialBasis] Using absolute cutoff of 12.0 Angstroms.
CRITICAL:root:Found the following scale factors: [('int_blocks.0.trip_interaction.scale_rbf', 'int_blocks.0.trip_interaction.scale_rbf'), ('int_blocks.0.trip_interaction.scale_cbf_sum', 'int_blocks.0.trip_interaction.scale_cbf_sum'), ('int_blocks.0.quad_interaction.scale_rbf', 'int_blocks.0.quad_interaction.scale_rbf'), ('int_blocks.0.quad_interaction.scale_cbf', 'int_blocks.0.quad_interaction.scale_cbf'), ('int_blocks.0.quad_interaction.scale_sbf_sum', 'int_blocks.0.quad_interaction.scale_sbf_sum'), ('int_blocks.0.atom_edge_interaction.scale

MisconfigurationException: No validation dataset found. If you do not want to use a validation dataset, set `validation_split` to 'disable' in the data module configuration.

In [ ]:
potential = model.potential()
print(potential)

calculator = model.ase_calculator()
print(calculator)


In [4]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

print(potential.predict([atoms], model.hparams.properties))

INFO:lightning_fabric.utilities.rank_zero:Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:lightning_fabric.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


DEBUG:fsspec.local:open file: /workspaces/MatterTune/examples/lightning_logs/version_18/hparams.yaml
/opt/conda/envs/jmp-peft/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:ro

[{'energy': tensor[1] [2.674], 'forces': tensor[3, 3] n=9 x∈[-2.978, 3.067] μ=0.067 σ=1.571 [[0.001, 1.065, 3.067], [0.000, 0.096, -2.978], [0.000, -0.603, -0.044]], 'stresses': tensor[1, 3, 3] n=9 x∈[-0.517, 3.190e-07] μ=-0.166 σ=0.189 [[[-0.517, 3.190e-07, -1.779e-06], [3.190e-07, -0.339, -0.161], [-1.779e-06, -0.161, -0.315]]]}]


In [5]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

# Set the calculator
atoms.calc = calculator

# Calculate the energy
energy = atoms.get_potential_energy()
print(energy)

INFO:lightning_fabric.utilities.rank_zero:Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:lightning_fabric.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
DEBUG:fsspec.local:open file: /workspaces/MatterTune/examples/lightning_logs/version_19/hparams.yaml


Atoms(symbols='H2O', pbc=True, cell=[10.0, 10.0, 10.0])


Predicting: |          | 0/? [00:00<?, ?it/s]

DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:ro

2.673945188522339


In [6]:
potential = model.potential()
atoms_1 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms_2 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms = [atoms_1, atoms_2]
predictions = potential.predict(atoms, ["energy", "forces"])
print("ase.Atoms 1 energy:", predictions[0]["energy"])
print("ase.Atoms 1 forces:", predictions[0]["forces"])
print("ase.Atoms 2 energy:", predictions[1]["energy"])
print("ase.Atoms 2 forces:", predictions[1]["forces"])

INFO:lightning_fabric.utilities.rank_zero:Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:lightning_fabric.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
DEBUG:fsspec.local:open file: /workspaces/MatterTune/examples/lightning_logs/version_20/hparams.yaml


Predicting: |          | 0/? [00:00<?, ?it/s]

DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:root:Observer not initialized but self.observe() called
DEBUG:ro

ase.Atoms 1 energy: tensor[1] [2.674]
ase.Atoms 1 forces: tensor[3, 3] n=9 x∈[-2.978, 3.067] μ=0.067 σ=1.571 [[0.001, 1.065, 3.067], [0.000, 0.096, -2.978], [0.000, -0.603, -0.044]]
ase.Atoms 2 energy: tensor[1] [2.674]
ase.Atoms 2 forces: tensor[3, 3] n=9 x∈[-2.978, 3.067] μ=0.067 σ=1.571 [[0.001, 1.065, 3.067], [0.000, 0.096, -2.978], [0.000, -0.603, -0.044]]
